<a href="https://colab.research.google.com/github/Jsgithubchannel/deep_learning_practice/blob/main/situation_model_changed_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 1. 코랩 환경 설정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [26]:
#Kobert 파일 로드 from github
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-dpso9rt0
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-dpso9rt0


In [27]:
# 환경 설정 (필요한 모듈 import)
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [28]:
#GPU 사용 시
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. 데이터셋 전처리
import pandas as pd
chat_data = pd.read_excel('/content/drive/MyDrive/capstone_design/reduced_situation_dataset.xlsx')

In [ ]:
print(len(chat_data)) # 데이터 개수
chat_data.sample(n=10) # 랜덤 데이터 10개 출력

49660


,situation,sentence1,sentence2,sentence3
12770,대인관계,천식 환자들 모임에서 만난 사람이 천식에 좋다는 영양제를 자꾸 파는데 사기꾼 같아서...,아무리 생각해도 천식에 좋은 영양제라고 하기엔 효과가 별로 없을 거 같아.,이 약에 대해 인터넷에 자세히 검색해봐야겠어.
41376,"진로, 취업, 직장",이번에 지원한 기업에 또 서류탈락을 해서 너무 속상해.,사실 취업이 쉽지 않다보니 전공과 상관없는 곳에 지원했었어. 아마 다른 사람에 비해...,사실 잘 모르겠어. 워낙 요즘 취업이 어렵다 보니 아무곳이나 빨리 자리 잡았으면 해.
8060,"건강, 죽음",늙은 나도 살아가는데 아들이 사업이 잘 안 된다며 죽고 싶다는 말을 했어.,사업이 잘 안 된다고 해서 인생이 끝나는 건 아닌데 걱정이 돼.,아들에게 늙은 나도 살아간다며 용기를 주고 싶어.
11114,대인관계,운동 삼아 게이트볼을 시작했는데 비싼 장비로 열을 올리는 애들이 있더라. 답답해.,왜 그렇게 돈 낭비를 하는지 모르겠어. 걔네랑 게이트볼 하기 싫어졌어.,친구들한테 농담 반 진담 반으로 얘기해보는 게 좋겠어.
20869,"대인관계(부부, 자녀)",아내는 내가 주말에 일부러 늦게 일어난다고 생각해.,맞아. 내가 일부러 그런게 아닌데.,아직 안해봤어. 평일에 야근하고 피곤하다고 말해 봐야겠다.
32543,"재정, 은퇴, 노후준비",노후준비를 위해 가장 중요한 게 뭔지 알았어.,응. 건강이 최고라고 하는군. 난 꾸준하게 운동을 해서 신체 나이가 실제 나이보다 ...,적어도 일주일에 사 일 이상 운동했어. 삼십 분간 빨리 걷기 근력 운동으로 하체 ...
32142,"재정, 은퇴, 노후준비",남편이 노후자금을 마음대로 관리해서 배신당한 기분이야.,응. 생활비만 줘서 섭섭해.,큰 실패는 없어.
9426,"건강, 죽음",나이가 많아지면서 죽음이 가까워오는 것을 느껴. 두렵고 불안해.,언제 이렇게 나이가 들어 버렸는지 모르겠어. 죽고 싶지 않은데 정말 두려워.,남편과 나가서 기분 전환을 해야겠어.
7131,"건강, 죽음",심장마비로 남편을 떠나 보내고 매일 아침이면 눈을 뜨기가 두려워.,차갑게 식어버렸던 남편의 몸을 생각하면 지금도 소름이 돋고 눈물이 나곤 해.,이제 혼자서는 못 살 것 같아. 아무래도 딸네 집에서 지내다 와야겠어.
9455,"건강, 죽음",나는 아직 건강한데 밖에 나가면 노인 취급을 해.,버스에서 글쎄 나더러 할아버지라면서 노약좌석을 양보하더군.,난 아직 늙지않았다고 나도 모르게 흥분을 해서 소리를 질렀어.


In [ ]:
situations = ['가족관계','학업 및 진로','학교폭력/따돌림','대인관계(부부, 자녀)','대인관계','연애, 결혼, 출산', '진로, 취업, 직장', '재정, 은퇴, 노후준비', '건강, 죽음', '건강', '직장, 업무 스트레스', '재정'] # 감정 대분류
for i in range(len(situations)):
  chat_data.loc[(chat_data['situation'] == situations[i]), 'situation'] = i

In [ ]:
data_list = []
for label, sentence1, sentence2, sentence3 in zip(chat_data['situation'], chat_data['sentence1'], chat_data['sentence2'], chat_data['sentence3']):
    data = []
    sentence = sentence1+' '+sentence2+' '+sentence3
    data.append(sentence)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[49000]

['공부가 내 길이 아니라 공부를 안하는 것 뿐이야. 공부가 내 길이 아니라 가수가 내 길이라서 안하는거 뿐이야. 난 노래 연습을 할거야 그거면 돼.',
 '1']

In [ ]:
# 3. Train data & Test data
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
# 4. KoBERT 입력 데이터로 만들기

# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [34]:
# Setting parameters
max_len = 200
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

39728
9932


In [ ]:
#토큰화
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

#BERT 모델, Vocabulary 불러오기
from kobert.pytorch_kobert import get_kobert_model
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file) 

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-10945drh/kobert-tokenizer_ed560bef6bf24c8bb4e19b7554d94b66
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-10945drh/kobert-tokenizer_ed560bef6bf24c8bb4e19b7554d94b66
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4648 sha256=89e6fa4b568315cd75f37a7b59fef98531ca90450433bde34a19e95e7ad18d7b
  Stored in directory: /tmp/pip-ephem-wheel-cache-0kl_ty86/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok,vocab,  max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 4162, 6356, 5468, 7234, 1846, 3111, 5398, 5859, 3135, 7828,
        4627, 5330, 1567, 3194, 5940, 6022, 2752, 5931, 3191, 3574, 6585,
        1852, 5760,  517, 5617, 7096, 3128,  517, 5499, 7350, 5663, 5782,
          54,   54, 2251, 7832, 3191, 1958, 6150, 2307, 5920, 6113, 6113,
        7784, 4966, 1562, 5330, 3105, 5850,  517, 5617, 6368, 3059, 6553,
        5100, 5671, 5702,   54, 3192, 5760, 4799, 7659, 6116,  517, 7375,
        5357, 6553, 1610, 2437, 7088,  775, 2251, 7788, 4435, 3135, 7811,
        4077, 6116, 2086, 6291, 5859, 1189, 4156, 7836, 5377, 6844,   54,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [ ]:
# torch 형식의 dataset
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# 5. KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=12, ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [35]:
# KoBERT 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/2483 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.0791397094726562 train acc 0.5625
epoch 1 batch id 201 loss 0.8637078404426575 train acc 0.6791044776119403
epoch 1 batch id 401 loss 0.6455494165420532 train acc 0.6893703241895262
epoch 1 batch id 601 loss 0.9878455400466919 train acc 0.706634775374376
epoch 1 batch id 801 loss 1.120129108428955 train acc 0.7116104868913857
epoch 1 batch id 1001 loss 0.7934266924858093 train acc 0.7185314685314685
epoch 1 batch id 1201 loss 1.2703098058700562 train acc 0.7216902581182348
epoch 1 batch id 1401 loss 0.4209461212158203 train acc 0.724214846538187
epoch 1 batch id 1601 loss 1.0402424335479736 train acc 0.7215412242348532
epoch 1 batch id 1801 loss 0.8193891644477844 train acc 0.7166157690172127
epoch 1 batch id 2001 loss 1.0049397945404053 train acc 0.7128935532233883
epoch 1 batch id 2201 loss 1.149207592010498 train acc 0.7092514766015448
epoch 1 batch id 2401 loss 0.4892946779727936 train acc 0.7062942523948355
epoch 1 train acc 0.704893274265002


  0%|          | 0/621 [00:00<?, ?it/s]

epoch 1 test acc 0.665089908749329


...

epoch 9 test acc 0.7028985507246377


  0%|          | 0/2483 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.34994471073150635 train acc 0.875
epoch 10 batch id 201 loss 0.2661716341972351 train acc 0.9259950248756219
epoch 10 batch id 401 loss 0.14968684315681458 train acc 0.9318890274314214
epoch 10 batch id 601 loss 0.03246423974633217 train acc 0.9362520798668885
epoch 10 batch id 801 loss 0.4312843382358551 train acc 0.9392946317103621
epoch 10 batch id 1001 loss 0.4475109577178955 train acc 0.9427447552447552
epoch 10 batch id 1201 loss 0.16907954216003418 train acc 0.9449417152373022
epoch 10 batch id 1401 loss 0.15528441965579987 train acc 0.945173090649536
epoch 10 batch id 1601 loss 0.3938988745212555 train acc 0.9428872579637726
epoch 10 batch id 1801 loss 0.06487198173999786 train acc 0.9407967795669073
epoch 10 batch id 2001 loss 0.05307812988758087 train acc 0.9382808595702149
epoch 10 batch id 2201 loss 0.3642066717147827 train acc 0.9339220808723308
epoch 10 batch id 2401 loss 0.11201546341180801 train acc 0.9301853394418992
epoch 10 train acc 0.9289

  0%|          | 0/621 [00:00<?, ?it/s]

epoch 10 test acc 0.683407139023081


In [41]:
#모델 전체 저장
path = F"/content/drive/MyDrive/capstone_design/all_model_epoch10.pth"
torch.save(model, path)

In [ ]:
#모델 전체 불러오기
path = F"/content/drive/MyDrive/capstone_design/all_model_epoch10.pth"

model.load_state_dict(torch.load(path))
model.eval()

In [55]:
# 저장하기
state = {
        'epoch': num_epochs,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'loss': loss,
}
path ="/content/drive/MyDrive/capstone_design/model_epoch10.pth"
torch.save(state,path)

In [ ]:
# 불러오기
checkpoint = torch.load("/content/drive/MyDrive/capstone_design/model_epoch10.pth")
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# model.train()

In [58]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("가족관계")
            elif np.argmax(logits) == 1:
                test_eval.append("학업 및 진로")
            elif np.argmax(logits) == 2:
                test_eval.append("학교폭력/따돌림")
            elif np.argmax(logits) == 3:
                test_eval.append("대인관계(부부, 자녀)")
            elif np.argmax(logits) == 4:
                test_eval.append("대인관계")
            elif np.argmax(logits) == 5:
                test_eval.append("연애, 결혼, 출산")
            elif np.argmax(logits) == 6:
                test_eval.append("진로, 취업, 직장")
            elif np.argmax(logits) == 7:
                test_eval.append("재정, 은퇴, 노후준비")
            elif np.argmax(logits) == 8:
                test_eval.append("건강, 죽음")
            elif np.argmax(logits) == 9:
                test_eval.append("건강")
            elif np.argmax(logits) == 10:
                test_eval.append("직장, 업무 스트레스")
            elif np.argmax(logits) == 11:
                test_eval.append("재정")

            print(logits)
        print(">> 입력하신 내용에서 " + test_eval[0] + " 상황입니다.")

In [59]:
#질문 무한반복하기! 0 입력시 종료

end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 공부가 내 길이 아니라 공부를 안하는 것 뿐이야. 공부가 내 길이 아니라 가수가 내 길이라서 안하는거 뿐이야. 난 노래 연습을 할거야 그거면 돼.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[-0.08626291  7.5850387   1.0644163   0.12527494  0.2832121  -2.4605815
  0.5631924  -0.43216223 -1.5626361   0.08037304 -2.4776974  -0.572239  ]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 공부는 힘들어
[-0.8184731   7.1750045   1.5238721   0.7509221   0.39652944 -2.6210127
 -0.0325031   0.01507865 -1.4918983   0.36574253 -2.4716053  -0.8044045 ]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 돈버는 것도 힘들어
[-2.5421252  -0.7501472  -1.6436696  -0.95584273  0.85087025 -2.2679324
  1.2098463   2.2372916  -1.0911473  -0.9288866  -0.34687385  5.5733123 ]
>> 입력하신 내용에서 재정 상황입니다.


하고싶은 말을 입력해주세요 : 삶의 의미는 뭘까?
[-0.1750377   2.8037302   0.6891271  -1.555724    0.370161   -1.8484235
  1.3590399   0.45635992  2.256129   -1.1854153  -2.4654038  -0.20953509]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 요즘 몸이 안좋아서 필라테스 다녀왔어
[ 1.230831   -1.0924592  -1.3921441  -2.224237    1.3496826  -0.09393229
 -2.1893322  -1.9079479   6.3059173   1.1123224  -2.4035513  -0.3818108 ]
>> 입력하신 내용에서 건강, 죽음 상황입니다.


하고싶은 말

In [ ]:
#캐시 삭제
import gc
gc.collect()
torch.cuda.empty_cache()